## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# ⚠️ Exception Handling with Middleware

## Industry Use Case: Market Data Service Recovery

This notebook demonstrates exception handling for unreliable external services.

| Feature | FSI Application |
|---------|-----------------|
| **Error Catching** | Handle market data API timeouts |
| **Graceful Degradation** | Provide fallback responses |
| **Error Logging** | Track service failures |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print("✅ Environment loaded")

In [ ]:
from collections.abc import Awaitable, Callable

from agent_framework import FunctionInvocationContext
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded")

## Define Unstable Market Data Service

In [ ]:
def get_stock_price(symbol: str) -> str:
    """Get stock price - simulates an unreliable market data service."""
    # Simulate a service timeout
    raise TimeoutError("Market data service request timed out")

print("✅ Tool defined: get_stock_price (will always timeout)")

## Define Exception Handling Middleware

In [ ]:
async def exception_handling_middleware(
    context: FunctionInvocationContext, 
    next: Callable[[FunctionInvocationContext], Awaitable[None]]
) -> None:
    """Middleware that catches exceptions and provides graceful fallbacks."""
    function_name = context.function.name

    try:
        print(f"[ExceptionHandler] Executing: {function_name}")
        await next(context)
        print(f"[ExceptionHandler] {function_name} completed successfully")
    except TimeoutError as e:
        print(f"[ExceptionHandler] Caught TimeoutError: {e}")
        # Override result with user-friendly message
        context.result = (
            "Market data service is temporarily unavailable.",
            "Please try again later or contact support.",
        )

print("✅ Middleware defined: exception_handling_middleware")

## Run Example

In [ ]:
async def main():
    print("=== Exception Handling Middleware Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="MarketDataAgent",
            instructions="You are a market data assistant. Use get_stock_price to fetch prices.",
            tools=get_stock_price,
            middleware=[exception_handling_middleware],
        ) as agent,
    ):
        query = "What is the current price of MSFT stock?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

await main()

## 📋 Key Takeaways

| Feature | Description |
|---------|-------------|
| **try-except** | Wrap `await next(context)` to catch errors |
| **context.result** | Override with user-friendly message |
| **Graceful degradation** | Agent responds helpfully even on failure |